# Goal: Make a map of Sangenjaya, Tokyo 

## 1. import libraries 

In [1]:
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd
from matplotlib.patches import Rectangle
import matplotlib.font_manager as fm

In [2]:
# Japanese font configuration
plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial Unicode MS', 'Hiragino Sans', 
                                     'MS Gothic', 'Yu Gothic']
plt.rcParams['axes.unicode_minus'] = False

def main():
    print("="*60)
    print("三軒茶屋 TYPOGRAPHY ANALYSIS")
    print("30 Day Map Challenge - Places")
    print("="*60)
    
    # 1. SETUP LOCATION
    print("\n[1/7] Setting up Sangenjaya Station location...")
    # Sangenjaya Station: 35.6433°N, 139.6700°E
    lat, lon = 35.6433, 139.6700
    station = gpd.GeoDataFrame(
        geometry=[Point(lon, lat)], 
        crs="EPSG:4326"
    )
    print(f"✓ Station coordinates: ({lat:.4f}°N, {lon:.4f}°E)")
    
    # 2. CREATE BUFFER
    print("\n[2/7] Creating 8km buffer around station...")
    buffer_meters = 8000
    station_buffer = station.to_crs(epsg=3857).buffer(buffer_meters).to_crs(epsg=4326)
    buffer_poly = gpd.GeoDataFrame(geometry=[station_buffer.iloc[0]], crs="EPSG:4326")
    print(f"✓ Buffer created: {buffer_meters/1000}km radius")
    
    # 3. FETCH OSM DATA
    print("\n[3/7] Fetching named features from OpenStreetMap...")
    print("This may take a few minutes depending on data density...")
    tags_name = {"name": True}
    try:
        names = ox.features_from_polygon(buffer_poly.geometry[0], tags=tags_name)
        print(f"✓ Retrieved {len(names):,} named features")
    except Exception as e:
        print(f"✗ Error fetching OSM data: {e}")
        print("Please check your internet connection and try again.")
        return
    
    # 4. FILTER FOR 三軒茶屋
    print("\n[4/7] Filtering for features with '三軒茶屋'...")
    sangenjaya_features = names[names['name'].str.contains('三軒茶屋', na=False)]
    print(f"✓ Found {len(sangenjaya_features)} features with 三軒茶屋")
    
    if len(sangenjaya_features) == 0:
        print("\n⚠ No features found with 三軒茶屋 in the name.")
        print("This might mean the search area needs adjustment.")
        return
    
    # 5. ANALYZE FEATURES
    print("\n[5/7] Analyzing feature types...")
    
    # Count geometry types
    geom_counts = sangenjaya_features.geometry.geom_type.value_counts()
    print("\nGeometry Types:")
    for geom, count in geom_counts.items():
        print(f"  {geom}: {count}")
    
    # Analyze feature tags
    feature_types = {}
    
    for tag in ['amenity', 'shop', 'building', 'railway', 'highway', 'office', 
                'leisure', 'tourism', 'historic']:
        if tag in sangenjaya_features.columns:
            counts = sangenjaya_features[tag].value_counts()
            if len(counts) > 0:
                feature_types[tag] = counts
                print(f"\n{tag.upper()}:")
                for value, count in counts.head(10).items():
                    print(f"  {value}: {count}")
    
    # List all features
    print("\n" + "="*60)
    print("ALL FEATURES NAMED 三軒茶屋:")
    print("="*60)
    
    for idx, row in sangenjaya_features.iterrows():
        tags = []
        for tag in ['amenity', 'shop', 'building', 'railway', 'highway', 'office']:
            if pd.notna(row.get(tag)):
                tags.append(f"{tag}:{row[tag]}")
        
        tag_str = ", ".join(tags) if tags else "untagged"
        geom_type = row.geometry.geom_type
        print(f"• {row['name']}")
        print(f"  Type: {tag_str}")
        print(f"  Geometry: {geom_type}")
        print()
    
    # 6. CREATE MAP
    print("[6/7] Creating map visualization...")
    fig, ax = plt.subplots(figsize=(16, 16))
    
    # Plot buffer boundary
    buffer_poly.boundary.plot(ax=ax, color='#cccccc', linewidth=2, 
                              linestyle='--', label='8km Buffer', zorder=1)
    
    # Get streets for context (smaller area to avoid too much data)
    try:
        print("  Adding street context...")
        small_buffer = station.to_crs(epsg=3857).buffer(2000).to_crs(epsg=4326)
        graph = ox.graph_from_polygon(small_buffer.iloc[0], network_type='drive')
        nodes, edges = ox.graph_to_gdfs(graph)
        edges.plot(ax=ax, linewidth=0.5, color='#e0e0e0', alpha=0.4, zorder=2)
    except:
        print("  (Skipping street network)")
    
    # Plot features by geometry type
    colors = {'Point': '#2E86AB', 'Polygon': '#A23B72', 'LineString': '#F18F01'}
    
    for geom_type in ['LineString', 'Polygon', 'Point']:
        subset = sangenjaya_features[sangenjaya_features.geometry.geom_type == geom_type]
        if len(subset) > 0:
            if geom_type == 'Point':
                subset.plot(ax=ax, color=colors[geom_type], markersize=150, 
                           alpha=0.8, edgecolor='white', linewidth=1,
                           label=f'三軒茶屋 {geom_type}s ({len(subset)})', 
                           zorder=5)
            elif geom_type == 'Polygon':
                subset.plot(ax=ax, color=colors[geom_type], alpha=0.4, 
                           edgecolor=colors[geom_type], linewidth=2,
                           label=f'三軒茶屋 {geom_type}s ({len(subset)})', 
                           zorder=4)
            elif geom_type == 'LineString':
                subset.plot(ax=ax, color=colors[geom_type], linewidth=3, alpha=0.7,
                           label=f'三軒茶屋 {geom_type}s ({len(subset)})', 
                           zorder=4)
    
    # Plot station with star
    station.plot(ax=ax, color='#C73E1D', markersize=400, marker='*', 
                edgecolor='white', linewidth=2,
                label='三軒茶屋駅 (Sangenjaya Station)', zorder=6)
    
    # Styling
    ax.set_title('三軒茶屋 (Sangenjaya) Typography Analysis\n30 Day Map Challenge — Places', 
                 fontsize=24, fontweight='bold', pad=30, 
                 family='sans-serif')
    ax.set_xlabel('Longitude (経度)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Latitude (緯度)', fontsize=14, fontweight='bold')
    
    # Legend
    ax.legend(loc='upper left', fontsize=11, framealpha=0.95, 
              edgecolor='black', fancybox=True)
    
    # Grid
    ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Stats box
    stats_text = (
        f"Total Features: {len(sangenjaya_features)}\n"
        f"Search Radius: {buffer_meters/1000}km\n"
        f"Location: Setagaya, Tokyo\n"
        f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}"
    )
    ax.text(0.98, 0.02, stats_text, transform=ax.transAxes, 
            fontsize=11, verticalalignment='bottom', horizontalalignment='right',
            bbox=dict(boxstyle='round,pad=0.8', facecolor='white', 
                     edgecolor='black', alpha=0.9, linewidth=1.5),
            family='monospace')
    
    # Add north arrow
    ax.annotate('N', xy=(0.05, 0.95), xytext=(0.05, 0.90),
                xycoords='axes fraction', fontsize=20, fontweight='bold',
                ha='center', va='center',
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))
    
    plt.tight_layout()
    
    # 7. SAVE OUTPUTS
    print("\n[7/7] Saving outputs...")
    
    # Save map
    map_path = 'sangenjaya_typography_map.png'
    plt.savefig(map_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"✓ Map saved: {map_path}")
    
    # Save GeoJSON
    geojson_path = 'sangenjaya_features.geojson'
    sangenjaya_features.to_file(geojson_path, driver='GeoJSON')
    print(f"✓ GeoJSON saved: {geojson_path}")
    
    # Save CSV summary
    csv_data = []
    for idx, row in sangenjaya_features.iterrows():
        csv_data.append({
            'name': row['name'],
            'geometry_type': row.geometry.geom_type,
            'amenity': row.get('amenity', ''),
            'shop': row.get('shop', ''),
            'building': row.get('building', ''),
            'railway': row.get('railway', ''),
            'highway': row.get('highway', '')
        })
    
    csv_path = 'sangenjaya_features.csv'
    pd.DataFrame(csv_data).to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"✓ CSV saved: {csv_path}")
    
    print("\n" + "="*60)
    print("✓ ANALYSIS COMPLETE!")
    print("="*60)
    print(f"\nFound {len(sangenjaya_features)} features with 三軒茶屋 in their name")
    print(f"within {buffer_meters/1000}km of Sangenjaya Station.")
    print("\nCheck the output files for detailed results!")
    
    plt.show()

if __name__ == "__main__":
    main()

三軒茶屋 TYPOGRAPHY ANALYSIS
30 Day Map Challenge - Places

[1/7] Setting up Sangenjaya Station location...
✓ Station coordinates: (35.6433°N, 139.6700°E)

[2/7] Creating 8km buffer around station...
✓ Buffer created: 8.0km radius

[3/7] Fetching named features from OpenStreetMap...
This may take a few minutes depending on data density...
✗ Error fetching OSM data: ('Connection broken: IncompleteRead(82 bytes read, 8014 more expected)', IncompleteRead(82 bytes read, 8014 more expected))
Please check your internet connection and try again.
